In [6]:
import requests, json
from time import sleep

In [7]:
# Secret token
token = open("ACCESS_TOKEN.txt", "r").read()

In [16]:
# constant values.
BASE_URL = "https://api.genius.com"
CLIENT_ACCESS_TOKEN = token
QUERY_SIZE=1000

In [9]:
# send request and get response in json format.
def _get(path, params=None, headers=None):

    # generate request URL
    requrl = '/'.join([BASE_URL, path])
    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}

    response = requests.get(url=requrl, params=params, headers=headers)
    response.raise_for_status()

    return response.json()

In [10]:
# read in list of artist names
with open("artistNames.txt", "r") as f:
    artists = f.readlines()
    
artists = [x.strip() for x in artists]
print(str(len(artists))+" artists total")

12175 artists total


In [15]:
# query only a subset of artists at a time to not overwork server
# increment chunk_num on each run
chunk_num = 0 
artists_chunk = artists[QUERY_SIZE*chunk_num:QUERY_SIZE*(chunk_num+1)]

# write (artist_name,artist_id) pairs to artists.txt file
with open("artists.txt", "a") as f:

    # iterate over artist names
    for artist in artists_chunk:

        # get artist_id
        find_id = _get("search", {'q': artist})

        # write to file
        f.write(str(artist_id)+","+artist+"\n")

        sleep(.25)

HTTPError: 401 Client Error: Unauthorized for url: https://api.genius.com/search?q=beyonce-knowles